In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
import tensorflow_hub as hub
import pandas as pd

# 載入資料

In [2]:
# 將訓練資料拆分成7:3，分別是訓練與測試集，並且取得Info
[train_ds, valid_ds, test_ds], info = tfds.load('colorectal_histology',
                            split=['train[:70%]','train[70%:85%]', 'train[85%:]'],
                            shuffle_files=True,
                            with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/colorectal_histology/2.0.0.incompleteJISXZA/colorectal_histology-train.tfr…

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [3]:
# 參數設定
#input_shapes = info.features[info.supervised_keys[0]].shape
input_shapes = (224,224,3)
img_size = input_shapes[:2]
batch_size = 32
epoch = 50

In [4]:
# 預處理函數
def preprocess_data(data):
    image = data['image']
    label = data['label']
    # 將圖片大小調整為指定大小,並正規化像素值至 [0, 1]
    image = tf.image.resize(image, img_size) / 255.0
    return image, label

# 將預處理函數應用到數據集，並將資料分批
train_data = train_ds.map(preprocess_data).batch(batch_size)
valid_data = valid_ds.map(preprocess_data).batch(batch_size)
test_data = test_ds.map(preprocess_data).batch(batch_size)

# EarlyStopping設定
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 定義模型

In [5]:
# 定義 CNN 模型 (用於callbacks)
model_1 = models.Sequential()
model_1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shapes))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(512, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Flatten())
model_1.add(layers.Dense(256, activation='relu'))
model_1.add(layers.Dense(128, activation='relu'))
model_1.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_1.summary()
# 編譯模型
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [6]:
# 使用 TensorFlow Hub 中的 MobileNetV2 預訓練模型
MNV2_model = hub.load("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4")
# 凍結預訓練模型的權重
MNV2_model.trainable = False

model_2 = models.Sequential()
model_2.add(hub.KerasLayer(MNV2_model, trainable=False, input_shape=input_shapes))
model_2.add(layers.Flatten())
model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_2.summary()
# 編譯模型
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              3540265   
                                                                 
 flatten_1 (Flatten)         (None, 1001)              0         
                                                                 
 dense_3 (Dense)             (None, 128)               128256    
                                                                 
 dense_4 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 3669553 (14.00 MB)
Trainable params: 129288 (505.03 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________


In [7]:
# 建立 VGG19 模型
VGG19_model = tf.keras.applications.VGG19(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 VGG19 的權重
VGG19_model.trainable = False

model_3 = models.Sequential(VGG19_model)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(256, activation='relu'))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_3.summary()
# 編譯模型
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

80134624/80134624 [==============================] - 3s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_5 (Dense)             (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 8)                 2056      
                                                                 
Total params: 26449224 (100.90 MB)
Trainable params: 6424840 (24.51 MB)
Non-trainable params: 20024384 (76.39 MB)
_______

In [8]:
# 建立 DenseNet121 模型
DN121_model = tf.keras.applications.DenseNet121(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 DenseNet121 的權重
DN121_model.trainable = False

model_4 = models.Sequential(DN121_model)
model_4.add(layers.GlobalAveragePooling2D())
model_4.add(layers.Dense(256, activation='relu'))
model_4.add(layers.Dropout(0.5))
model_4.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_4.summary()
# 編譯模型
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

29084464/29084464 [==============================] - 1s 0us/step
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_7 (Dense)             (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 8)                 2056      
                                                                 
Total params: 7301960 (27.85 MB)
Trainable params: 2644

# 訓練模型

In [9]:
# 使用 CNN

history_1 = model_1.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 17s 85ms/step - loss: 1.4875 - accuracy: 0.3409 - val_loss: 1.0663 - val_accuracy: 0.4973
Epoch 2/50
110/110 [==============================] - 6s 57ms/step - loss: 0.9956 - accuracy: 0.5423 - val_loss: 0.9911 - val_accuracy: 0.5827
Epoch 3/50
110/110 [==============================] - 8s 74ms/step - loss: 0.8691 - accuracy: 0.6463 - val_loss: 0.8359 - val_accuracy: 0.7000
Epoch 4/50
110/110 [==============================] - 6s 57ms/step - loss: 0.7705 - accuracy: 0.6909 - val_loss: 0.6851 - val_accuracy: 0.7293
Epoch 5/50
110/110 [==============================] - 7s 65ms/step - loss: 0.7031 - accuracy: 0.7283 - val_loss: 0.8333 - val_accuracy: 0.6773
Epoch 6/50
110/110 [==============================] - 6s 57ms/step - loss: 0.6764 - accuracy: 0.7249 - val_loss: 0.6741 - val_accuracy: 0.7427
Epoch 7/50
110/110 [==============================] - 7s 66ms/step - loss: 0.6308 - accuracy: 0.7523 - val_loss: 0.5977 - val_accuracy: 0.774

In [10]:
# 使用TFhub中 MobileNetV2 model
history_2 = model_2.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 17s 83ms/step - loss: 0.5720 - accuracy: 0.8031 - val_loss: 0.3995 - val_accuracy: 0.8813
Epoch 2/50
110/110 [==============================] - 6s 51ms/step - loss: 0.2776 - accuracy: 0.9011 - val_loss: 0.3588 - val_accuracy: 0.8960
Epoch 3/50
110/110 [==============================] - 7s 60ms/step - loss: 0.1896 - accuracy: 0.9369 - val_loss: 0.3680 - val_accuracy: 0.8907
Epoch 4/50
110/110 [==============================] - 6s 55ms/step - loss: 0.1357 - accuracy: 0.9557 - val_loss: 0.3939 - val_accuracy: 0.8827
Epoch 5/50
110/110 [==============================] - 6s 56ms/step - loss: 0.0968 - accuracy: 0.9694 - val_loss: 0.4140 - val_accuracy: 0.8787


In [11]:
# 使用 VGG19
history_3 = model_3.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 43s 277ms/step - loss: 1.4244 - accuracy: 0.5517 - val_loss: 0.7741 - val_accuracy: 0.7827
Epoch 2/50
110/110 [==============================] - 22s 199ms/step - loss: 0.8096 - accuracy: 0.6871 - val_loss: 0.5754 - val_accuracy: 0.8213
Epoch 3/50
110/110 [==============================] - 20s 185ms/step - loss: 0.6586 - accuracy: 0.7537 - val_loss: 0.5326 - val_accuracy: 0.8333
Epoch 4/50
110/110 [==============================] - 22s 200ms/step - loss: 0.6023 - accuracy: 0.7806 - val_loss: 0.4484 - val_accuracy: 0.8480
Epoch 5/50
110/110 [==============================] - 22s 202ms/step - loss: 0.5427 - accuracy: 0.8040 - val_loss: 0.4644 - val_accuracy: 0.8373
Epoch 6/50
110/110 [==============================] - 22s 198ms/step - loss: 0.5397 - accuracy: 0.7963 - val_loss: 0.5343 - val_accuracy: 0.8133
Epoch 7/50
110/110 [==============================] - 22s 200ms/step - loss: 0.5131 - accuracy: 0.8014 - val_loss: 0.4408 - val_ac

In [12]:
# 使用 DenseNet121
history_4 = model_4.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 31s 188ms/step - loss: 0.7888 - accuracy: 0.7186 - val_loss: 0.4347 - val_accuracy: 0.8587
Epoch 2/50
110/110 [==============================] - 13s 118ms/step - loss: 0.4302 - accuracy: 0.8503 - val_loss: 0.3557 - val_accuracy: 0.8773
Epoch 3/50
110/110 [==============================] - 13s 117ms/step - loss: 0.3464 - accuracy: 0.8800 - val_loss: 0.3270 - val_accuracy: 0.8907
Epoch 4/50
110/110 [==============================] - 13s 119ms/step - loss: 0.3129 - accuracy: 0.8923 - val_loss: 0.3073 - val_accuracy: 0.9053
Epoch 5/50
110/110 [==============================] - 13s 115ms/step - loss: 0.2813 - accuracy: 0.9066 - val_loss: 0.2888 - val_accuracy: 0.9080
Epoch 6/50
110/110 [==============================] - 13s 116ms/step - loss: 0.2466 - accuracy: 0.9157 - val_loss: 0.2774 - val_accuracy: 0.9107
Epoch 7/50
110/110 [==============================] - 13s 116ms/step - loss: 0.2267 - accuracy: 0.9251 - val_loss: 0.3073 - val_ac

# 儲存模型權重

In [13]:
# 最終以使用DenseNet121權重作為網頁式直腸癌識別模型
model_4.save_weights('DenseNet121_model_weights.h5')
# 加載模型權重
# model_4.load_weights('DenseNet121_model_weights.h5')

# 評估模型

In [14]:
model_list = [model_1, model_2, model_3, model_4]
result = pd.DataFrame(columns=['Accuracy', 'Loss'])

for model in model_list:
  test_loss, test_acc = model.evaluate(test_data)
  result = pd.concat([result, pd.DataFrame({'Accuracy': [test_acc], 'Loss': [test_loss]})], ignore_index=True)

24/24 [==============================] - 2s 91ms/step - loss: 0.2006 - accuracy: 0.9267


In [15]:
result.index = ['CNN(有callbacks)', 'MobileNetV2', 'VGG19', 'DenseNet121']

result

,Accuracy,Loss
CNN(有callbacks),0.797333,0.516011
MobileNetV2,0.888000,0.299963
VGG19,0.860000,0.401729
DenseNet121,0.926667,0.200575
